Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Benchmark Fairness Experiments for Equal Opportunity

Requires paths to the appropriate datasets.

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cPickle


## Load dataset

In [0]:
#@title Load Adult dataset

CATEGORICAL_COLUMNS = [
    'workclass', 'education', 'marital_status', 'occupation', 'relationship',
    'race', 'gender', 'native_country'
]
CONTINUOUS_COLUMNS = [
    'age', 'capital_gain', 'capital_loss', 'hours_per_week', 'education_num'
]
COLUMNS = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'gender',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
    'income_bracket'
]
LABEL_COLUMN = 'label'

PROTECTED_GROUPS = [
    'gender_Female', 'gender_Male', 'race_White', 'race_Black'
]


def get_adult_data():
  train_file = PATH_TO_ADULT_TRAIN_FILE
  test_file = PATH_TO_ADULT_TEST_FILE

  train_df_raw = pd.read_csv(train_file, names=COLUMNS, skipinitialspace=True)
  test_df_raw = pd.read_csv(
      test_file, names=COLUMNS, skipinitialspace=True, skiprows=1)

  train_df_raw[LABEL_COLUMN] = (
      train_df_raw['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)
  test_df_raw[LABEL_COLUMN] = (
      test_df_raw['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)
  # Preprocessing Features
  pd.options.mode.chained_assignment = None  # default='warn'

  # Functions for preprocessing categorical and continuous columns.
  def binarize_categorical_columns(input_train_df,
                                   input_test_df,
                                   categorical_columns=[]):

    def fix_columns(input_train_df, input_test_df):
      test_df_missing_cols = set(input_train_df.columns) - set(
          input_test_df.columns)
      for c in test_df_missing_cols:
        input_test_df[c] = 0
      train_df_missing_cols = set(input_test_df.columns) - set(
          input_train_df.columns)
      for c in train_df_missing_cols:
        input_train_df[c] = 0
      input_train_df = input_train_df[input_test_df.columns]
      return input_train_df, input_test_df

    # Binarize categorical columns.
    binarized_train_df = pd.get_dummies(
        input_train_df, columns=categorical_columns)
    binarized_test_df = pd.get_dummies(
        input_test_df, columns=categorical_columns)
    # Make sure the train and test dataframes have the same binarized columns.
    fixed_train_df, fixed_test_df = fix_columns(binarized_train_df,
                                                binarized_test_df)
    return fixed_train_df, fixed_test_df
  
  def bucketize_continuous_column(input_train_df,
                                  input_test_df,
                                  continuous_column_name,
                                  num_quantiles=None,
                                  bins=None):
    assert (num_quantiles is None or bins is None)
    if num_quantiles is not None:
      train_quantized, bins_quantized = pd.qcut(
          input_train_df[continuous_column_name],
          num_quantiles,
          retbins=True,
          labels=False)
      input_train_df[continuous_column_name] = pd.cut(
          input_train_df[continuous_column_name], bins_quantized, labels=False)
      input_test_df[continuous_column_name] = pd.cut(
          input_test_df[continuous_column_name], bins_quantized, labels=False)
    elif bins is not None:
      input_train_df[continuous_column_name] = pd.cut(
          input_train_df[continuous_column_name], bins, labels=False)
      input_test_df[continuous_column_name] = pd.cut(
          input_test_df[continuous_column_name], bins, labels=False)

  # Filter out all columns except the ones specified.
  train_df = train_df_raw[CATEGORICAL_COLUMNS + CONTINUOUS_COLUMNS +
                          [LABEL_COLUMN]]
  test_df = test_df_raw[CATEGORICAL_COLUMNS + CONTINUOUS_COLUMNS +
                        [LABEL_COLUMN]]
  # Bucketize continuous columns.
  bucketize_continuous_column(train_df, test_df, 'age', num_quantiles=4)
  bucketize_continuous_column(
      train_df, test_df, 'capital_gain', bins=[-1, 1, 4000, 10000, 100000])
  bucketize_continuous_column(
      train_df, test_df, 'capital_loss', bins=[-1, 1, 1800, 1950, 4500])
  bucketize_continuous_column(
      train_df, test_df, 'hours_per_week', bins=[0, 39, 41, 50, 100])
  bucketize_continuous_column(
      train_df, test_df, 'education_num', bins=[0, 8, 9, 11, 16])
  train_df, test_df = binarize_categorical_columns(
      train_df,
      test_df,
      categorical_columns=CATEGORICAL_COLUMNS + CONTINUOUS_COLUMNS)
  feature_names = list(train_df.keys())
  feature_names.remove(LABEL_COLUMN)
  num_features = len(feature_names)
  return train_df, test_df, feature_names


train_df, test_df, feature_names = get_adult_data()
X_train_adult = np.array(train_df[feature_names])
y_train_adult = np.array(train_df[LABEL_COLUMN])
X_test_adult = np.array(test_df[feature_names])
y_test_adult = np.array(test_df[LABEL_COLUMN])

protected_train_adult = [np.array(train_df[g]) for g in PROTECTED_GROUPS]
protected_test_adult = [np.array(test_df[g]) for g in PROTECTED_GROUPS]


In [0]:
#@title Load Bank dataset

FEATURES = [
    u'campaign', u'previous', u'emp.var.rate', u'cons.price.idx',
    u'cons.conf.idx', u'euribor3m', u'nr.employed', u'job_admin.',
    u'job_blue-collar', u'job_entrepreneur', u'job_housemaid',
    u'job_management', u'job_retired', u'job_self-employed', u'job_services',
    u'job_student', u'job_technician', u'job_unemployed', u'job_unknown',
    u'marital_divorced', u'marital_married', u'marital_single',
    u'marital_unknown', u'education_basic.4y', u'education_basic.6y',
    u'education_basic.9y', u'education_high.school', u'education_illiterate',
    u'education_professional.course', u'education_university.degree',
    u'education_unknown', u'default_no', u'default_unknown', u'default_yes',
    u'housing_no', u'housing_unknown', u'housing_yes', u'loan_no',
    u'loan_unknown', u'loan_yes', u'contact_cellular', u'contact_telephone',
    u'day_of_week_fri', u'day_of_week_mon', u'day_of_week_thu',
    u'day_of_week_tue', u'day_of_week_wed', u'poutcome_failure',
    u'poutcome_nonexistent', u'poutcome_success', u'y_yes', u'age_0', u'age_1',
    u'age_2', u'age_3', u'age_4', u'duration_0.0', u'duration_1.0',
    u'duration_2.0', u'duration_3.0', u'duration_4.0'
]
features = [
    u'campaign', u'previous', u'emp.var.rate', u'cons.price.idx',
    u'cons.conf.idx', u'euribor3m', u'nr.employed', u'job_admin.',
    u'job_blue-collar', u'job_entrepreneur', u'job_housemaid',
    u'job_management', u'job_retired', u'job_self-employed', u'job_services',
    u'job_student', u'job_technician', u'job_unemployed', u'job_unknown',
    u'marital_divorced', u'marital_married', u'marital_single',
    u'marital_unknown', u'education_basic.4y', u'education_basic.6y',
    u'education_basic.9y', u'education_high.school', u'education_illiterate',
    u'education_professional.course', u'education_university.degree',
    u'education_unknown', u'default_no', u'default_unknown', u'default_yes',
    u'housing_no', u'housing_unknown', u'housing_yes', u'loan_no',
    u'loan_unknown', u'loan_yes', u'contact_cellular', u'contact_telephone',
    u'day_of_week_fri', u'day_of_week_mon', u'day_of_week_thu',
    u'day_of_week_tue', u'day_of_week_wed', u'poutcome_failure',
    u'poutcome_nonexistent', u'poutcome_success', u'age_0', u'age_1',
    u'age_2', u'age_3', u'age_4', u'duration_0.0', u'duration_1.0',
    u'duration_2.0', u'duration_3.0', u'duration_4.0'
] 
LABEL_COLUMN = ["y_yes"]
protected_features = ['age_0', 'age_1', 'age_2', 'age_3', 'age_4']


def get_data():
  data_path = PATH_TO_BANK_DATA
  df = pd.read_csv(data_file, sep=';')
  continuous_features = [
      'campaign', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
      'euribor3m', 'nr.employed'
  ]
  continuous_to_categorical_features = ['age', 'duration']
  categorical_features = [
      'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
      'day_of_week', 'poutcome', 'y'
  ]

  # Functions for preprocessing categorical and continuous columns.
  def binarize_categorical_columns(input_df, categorical_columns=[]):
    # Binarize categorical columns.
    binarized_df = pd.get_dummies(input_df, columns=categorical_columns)
    return binarized_df

  def bucketize_continuous_column(input_df, continuous_column_name, bins=None):
    input_df[continuous_column_name] = pd.cut(
        input_df[continuous_column_name], bins, labels=False)

  for c in continuous_to_categorical_features:
    b = [0] + list(np.percentile(df[c], [20, 40, 60, 80, 100]))
    bucketize_continuous_column(df, c, bins=b)

  df = binarize_categorical_columns(
      df,
      categorical_columns=categorical_features +
      continuous_to_categorical_features)

  to_fill = [
      u'duration_0.0', u'duration_1.0', u'duration_2.0', u'duration_3.0',
      u'duration_4.0'
  ]
  for i in range(len(to_fill) - 1):
    df[to_fill[i]] = df[to_fill[i:]].max(axis=1)

  normalize_features = [
      'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'
  ]
  for feature in normalize_features:
    df[feature] = df[feature] - np.mean(df[feature])

  label = ["u'y_yes"]
  df = df[FEATURES]

  return df


df = get_data()

y = np.array(df[LABEL_COLUMN]).flatten()

X_train_bank, X_test_bank, y_train_bank, y_test_bank = train_test_split(df, y, test_size=0.2, random_state=42)
protected_train_bank = [X_train_bank[g] for g in protected_features]
protected_test_bank = [X_test_bank[g] for g in protected_features]
X_train_bank = np.array(X_train_bank[features])
X_test_bank = np.array(X_test_bank[features])



In [0]:
#@title Load COMPAS dataset

LABEL_COLUMN = 'two_year_recid'
PROTECTED_GROUPS = [
    'sex_Female', 'sex_Male', 'race_Caucasian', 'race_African-American'
]


def get_data():
  data_path = PATH_TO_COMPAS_DATA
  df = pd.read_csv(data_path)
  FEATURES = [
      'age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex',
      'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid',
      'two_year_recid'
  ]
  df = df[FEATURES]
  df = df[df.days_b_screening_arrest <= 30]
  df = df[df.days_b_screening_arrest >= -30]
  df = df[df.is_recid != -1]
  df = df[df.c_charge_degree != 'O']
  df = df[df.score_text != 'N/A']
  continuous_features = [
      'priors_count', 'days_b_screening_arrest', 'is_recid', 'two_year_recid'
  ]
  continuous_to_categorical_features = ['age', 'decile_score', 'priors_count']
  categorical_features = ['c_charge_degree', 'race', 'score_text', 'sex']

  # Functions for preprocessing categorical and continuous columns.
  def binarize_categorical_columns(input_df, categorical_columns=[]):
    # Binarize categorical columns.
    binarized_df = pd.get_dummies(input_df, columns=categorical_columns)
    return binarized_df

  def bucketize_continuous_column(input_df, continuous_column_name, bins=None):
    input_df[continuous_column_name] = pd.cut(
        input_df[continuous_column_name], bins, labels=False)

  for c in continuous_to_categorical_features:
    b = [0] + list(np.percentile(df[c], [20, 40, 60, 80, 90, 100]))
    if c == 'priors_count':
      b = list(np.percentile(df[c], [0, 50, 70, 80, 90, 100]))
    bucketize_continuous_column(df, c, bins=b)

  df = binarize_categorical_columns(
      df,
      categorical_columns=categorical_features +
      continuous_to_categorical_features)

  to_fill = [
      u'decile_score_0', u'decile_score_1', u'decile_score_2',
      u'decile_score_3', u'decile_score_4', u'decile_score_5'
  ]
  for i in range(len(to_fill) - 1):
    df[to_fill[i]] = df[to_fill[i:]].max(axis=1)
  to_fill = [
      u'priors_count_0.0', u'priors_count_1.0', u'priors_count_2.0',
      u'priors_count_3.0', u'priors_count_4.0'
  ]
  for i in range(len(to_fill) - 1):
    df[to_fill[i]] = df[to_fill[i:]].max(axis=1)

  features = [
      u'days_b_screening_arrest', u'c_charge_degree_F', u'c_charge_degree_M',
      u'race_African-American', u'race_Asian', u'race_Caucasian',
      u'race_Hispanic', u'race_Native American', u'race_Other',
      u'score_text_High', u'score_text_Low', u'score_text_Medium',
      u'sex_Female', u'sex_Male', u'age_0', u'age_1', u'age_2', u'age_3',
      u'age_4', u'age_5', u'decile_score_0', u'decile_score_1',
      u'decile_score_2', u'decile_score_3', u'decile_score_4',
      u'decile_score_5', u'priors_count_0.0', u'priors_count_1.0',
      u'priors_count_2.0', u'priors_count_3.0', u'priors_count_4.0'
  ]
  label = ['two_year_recid']

  df = df[features + label]
  return df, features, label

df, feature_names, label_column = get_data()

from sklearn.utils import shuffle
df = shuffle(df, random_state=12345)
N = len(df)
train_df = df[:int(N * 0.66)]
test_df = df[int(N * 0.66):]

X_train_compas = np.array(train_df[feature_names])
y_train_compas = np.array(train_df[label_column]).flatten()
X_test_compas = np.array(test_df[feature_names])
y_test_compas = np.array(test_df[label_column]).flatten()

protected_train_compas = [np.array(train_df[g]) for g in PROTECTED_GROUPS]
protected_test_compas = [np.array(test_df[g]) for g in PROTECTED_GROUPS]

In [0]:
#@title Load Crime dataset

LABEL_COLUMN = 'label'

EXCLUDED_COLUMNS = [
    'state', 'county', 'community', 'communityname', 'ViolentCrimesPerPop'
]
PROTECTED_GROUPS = ['racepctblack_cat_low', 'racepctblack_cat_high',
                    'racePctAsian_cat_low', 'racePctAsian_cat_high',
                    'racePctWhite_cat_low', 'racePctWhite_cat_high',
                    'racePctHisp_cat_low','racePctHisp_cat_high']

def _dataframes(data_dir):
  """Returns the dataframes and feature names."""
  train_file = os.path.join(data_dir, 'train.csv')
  val_file = os.path.join(data_dir, 'val.csv')
  test_file = os.path.join(data_dir, 'test.csv')

  # Replace all missing feature values with the mean over the training set.
  feature_names = [
      name for name in train_df.keys()
      if name not in [LABEL_COLUMN] + EXCLUDED_COLUMNS
  ]
  for column in feature_names:
    train_mean = train_df[column].mean()
    train_df[column].fillna(train_mean, inplace=True)
    val_df[column].fillna(train_mean, inplace=True)
    test_df[column].fillna(train_mean, inplace=True)

  return train_df, val_df, test_df, feature_names

train_df, val_df, test_df, feature_names = _dataframes(PATH_TO_CRIME_DATA)
train_df = pd.concat((train_df, val_df))
X_train_crime = np.array(train_df[feature_names])
y_train_crime = np.array(train_df[LABEL_COLUMN]).flatten()
X_test_crime = np.array(test_df[feature_names])
y_test_crime = np.array(test_df[LABEL_COLUMN]).flatten()

protected_train_crime = [np.array(train_df[g]) for g in PROTECTED_GROUPS]
protected_test_crime = [np.array(test_df[g]) for g in PROTECTED_GROUPS]

In [0]:
#@title Load German Statlog dataset 


data_path = PATH_TO_GERMAN_STATLOG
with open(data_path, "rb") as fp:
    X = cPickle.load(fp)
    y = cPickle.load(fp)

# protected attribute is whether is age
X_train_german, X_test_german, y_train_german, y_test_german = train_test_split(X, y, test_size=0.33, random_state=42)
protected_train_german = [np.where(X_train_german[:, 9] <= 30, 1, 0)]
protected_test_german = [np.where(X_test_german[:, 9] <= 30, 1, 0)]

## Prepare Data

In [0]:

dataset_names = ["Adult", "Bank", "COMPAS", "Crime", "German Statlog"]

datas = [(X_train_adult, y_train_adult, X_test_adult, y_test_adult, protected_train_adult, protected_test_adult),
         (X_train_bank, y_train_bank, X_test_bank, y_test_bank, protected_train_bank, protected_test_bank),
         (X_train_compas, y_train_compas, X_test_compas, y_test_compas, protected_train_compas, protected_test_compas),
         (X_train_crime, y_train_crime, X_test_crime, y_test_crime, protected_train_crime, protected_test_crime),
         (X_train_german, y_train_german, X_test_german, y_test_german, protected_train_german, protected_test_german),]

In [0]:
#@title Helper Functions
def get_error_and_violations(y_pred, y, protected_attributes):
  acc = np.mean(y_pred != y)
  violations = []
  for p in protected_attributes:
    protected_idxs = np.where(np.logical_and(p > 0, y > 0))
    positive_idxs = np.where(y > 0)
    violations.append(np.mean(y_pred[positive_idxs]) - np.mean(y_pred[protected_idxs]))
  pairwise_violations = []
  for i in range(len(protected_attributes)):
    for j in range(i+1, len(protected_attributes)):
      protected_idxs = np.where(np.logical_and(protected_attributes[i] > 0, protected_attributes[j] > 0))
      if len(protected_idxs[0]) == 0:
        continue
      pairwise_violations.append(np.mean(y_pred) - np.mean(y_pred[protected_idxs]))
  return acc, violations, pairwise_violations

## Logistic Regression on original dataset

In [0]:
#@title Run experiment

from sklearn.linear_model import LogisticRegression
for dataset_idx, dataset_name in enumerate(dataset_names):
  print("Processing ", dataset_name)
  X_train, y_train, X_test, y_test, protected_train, protected_test = datas[dataset_idx]
  model = LogisticRegression()
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  y_pred_test = model.predict(X_test)

  acc, violations, pairwise_violations = get_error_and_violations(y_pred_train, y_train, protected_train)
  print("Train Accuracy", acc)
  print("Train Violation", max(np.abs(violations)), " \t\t All violations", violations)
  if len(pairwise_violations) > 0:
    print("Train Intersect Violations", max(np.abs(pairwise_violations)), " \t All violations", pairwise_violations)

  acc, violations, pairwise_violations = get_error_and_violations(y_pred_test, y_test, protected_test)
  print("Test Accuracy", acc)
  print("Test Violation", max(np.abs(violations)), " \t\t All violations", violations)
  if len(pairwise_violations) > 0:
    print("Test Intersect Violations", max(np.abs(pairwise_violations)), " \t All violations", pairwise_violations)
  print()
  print()

Processing  Adult
Train Accuracy 0.859770891557
Train Violation 0.117634167714  		 All violations [0.093056776546672548, -0.01646861896555496, -0.0067588222483615512, 0.11763416771380275]
Train Intersect Violations 0.158277923277  	 All violations [0.11361360164680066, 0.1582779232765294, -0.072077592314821026, 0.073860584934578546]
Test Accuracy 0.858485351023
Test Violation 0.119537965876  		 All violations [0.086236195210520283, -0.015626337584215944, -0.0076106310728074611, 0.11953796587617693]
Test Intersect Violations 0.158158349955  	 All violations [0.11115215201775935, 0.15815834995501074, -0.07108345283695916, 0.072908589919246203]


Processing  Bank
Train Accuracy 0.906798179059
Train Violation 0.119987259118  		 All violations [-0.038546536336591608, 0.06213278994735949, 0.080963480963480938, 0.1199872591176939, -0.10843584276420098]
Test Accuracy 0.905802379218
Test Violation 0.145225362872  		 All violations [-0.075956555983915763, 0.013190730837789655, 0.0948684350320501

## Data debiasing procedure

In [0]:
def debias_weights(original_labels, predicted, protected_attributes, multipliers):
  exponents = np.zeros(len(original_labels))
  for i, m in enumerate(multipliers):
    exponents -= m * protected_attributes[i]
  weights = np.exp(exponents)/ (np.exp(exponents) + np.exp(-exponents))
  #weights = np.where(predicted > 0, weights, 1 - weights)
  weights = np.where(original_labels > 0, 1 - weights, weights)
  return weights

## Our method

In [0]:
for dataset_idx, dataset_name in enumerate(dataset_names):
  print("Processing ", dataset_name)
  X_train, y_train, X_test, y_test, protected_train, protected_test = datas[dataset_idx]
  multipliers = np.zeros(len(protected_train))
  weights = np.array([1] * X_train.shape[0])
  learning_rate = 1.
  n_iters = 100
  for it in xrange(n_iters):
    model = LogisticRegression()

    model.fit(X_train, y_train, weights)
    y_pred_train = model.predict(X_train)

    weights = debias_weights(y_train, y_pred_train, protected_train, multipliers)

    acc, violations, pairwise_violations = get_error_and_violations(y_pred_train, y_train, protected_train)
    multipliers += learning_rate * np.array(violations)


    if (it + 1) % n_iters == 0:
      print(multipliers)
      y_pred_test = model.predict(X_test)
      acc, violations, pairwise_violations = get_error_and_violations(y_pred_train, y_train, protected_train)
      print("Train Accuracy", acc)
      print("Train Violation", max(np.abs(violations)), " \t\t All violations", violations)
      if len(pairwise_violations) > 0:
        print("Train Intersect Violations", max(np.abs(pairwise_violations)), " \t All violations", pairwise_violations)

      acc, violations, pairwise_violations = get_error_and_violations(y_pred_test, y_test, protected_test)
      print("Test Accuracy", acc)
      print("Test Violation", max(np.abs(violations)), " \t\t All violations", violations)
      if len(pairwise_violations) > 0:
        print("Test Intersect Violations", max(np.abs(pairwise_violations)), " \t All violations", pairwise_violations)
      print()
      print()

Processing  Adult
[ 0.21833614 -0.03863979 -0.01807924  0.23354414]
Train Accuracy 0.85900310187
Train Violation 0.00112902859052  		 All violations [-0.00026112713678849708, 4.6212683019186684e-05, 6.1482610411500715e-05, -0.0011290285905234398]
Train Intersect Violations 0.130279936614  	 All violations [0.091603198244022174, 0.13027993661365347, -0.056324075987816191, 0.037163506749669006]
Test Accuracy 0.855352865303
Test Violation 0.00922528521253  		 All violations [-0.0036806896004654144, 0.00066695542514572104, -0.0011747403993581651, 0.0092252852125257467]
Test Intersect Violations 0.123498634906  	 All violations [0.086740460457961896, 0.12349863490574281, -0.054071182050675848, 0.040328017047144199]


Processing  Bank
[-0.06107246  0.12154373  0.19560109  0.29810647 -0.28492557]
Train Accuracy 0.903915022762
Train Violation 0.0115456113539  		 All violations [-0.0077944542614504786, 0.011545611353906915, 0.00083697978434821296, 0.0018826330039144468, 9.0352172048946411e-05]
